In [1]:
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

In [4]:
from nest_asyncio import apply
apply()
from SystemTrading.TradingDataHub.MarketDataStorage import RecevierDataStorage
from asyncio import Queue, Event


dummy_queues = []
dummy_events = []
for _ in range(10):
    dummy_queues.append(Queue())
for _ in range(2):
    dummy_events.append(Event())

ins_receiver_storage = RecevierDataStorage(*dummy_queues, *dummy_events)


In [ ]:
for var in vars(ins_receiver_storage):
    if var.startswith("storage"):
        

queue_ticker
queue_trade
queue_minTicker
queue_depth
queue_aggTrade
queue_kline_ws
queue_execution_ws
queue_kline_fetcher
queue_orderbook_fetcher
queue_send_all_storage
event_stop_loop
event_start_exponential
stroage_ticker
storage_trade
storage_minTicker
storage_depth
storage_aggTrade
storage_kline_ws
storage_execution_ws
storage_kline_fetcher
storage_orderbook_fetcher


In [ ]:
for i in vars(RecevierDataStorage):
    print(i)

In [ ]:
RecevierDataStorage.__annotations__